# Finding data gaps

Often we want to find gaps in the data.
For example in energy analysis, we want to identify periods of energy unavailability which may be shown by a lack of data or by a prolonged period of zero voltage in a timeseries.

We will show two basic approaches

- Finding gaps in the reported data
- Identifying continuous stretches of data meeting some criteria

In [1]:
# finding gaps method

%matplotlib inline
import pandas as pd

# read in data and be sure to load dates properly
data = pd.read_csv('EVI0000111.csv', index_col=0, parse_dates=True)
data.head()

,Unnamed: 0,TYPE,VALUE
DT,,,
2016-01-01 00:00:00,0,KWH,26.871
2016-01-01 00:15:00,1,KWH,26.874
2016-01-01 00:30:00,2,KWH,26.877
2016-01-01 00:45:00,3,KWH,26.880
2016-01-01 01:00:00,4,KWH,26.882


The key analysis step here is to subtract the preceding time value from each time value.
This will give you an array of data with the differences between measurements.
If your data is perfect and consistent, these will all be at the sampling interval.
If there are data outages, there will be measurements at longer intervals.

The `diff(1)` method achieves this.
(Note that we have to use the `Series` code to get this to work.)

In [2]:
# take time deltas with diff function
time_deltas = pd.Series(data.index).diff(1)
time_deltas.head()

0        NaT
1   00:15:00
2   00:15:00
3   00:15:00
4   00:15:00
Name: DT, dtype: timedelta64[ns]

Once we have this long list (30 thousand entries) of time differences, we need a way to intepret the data in a way we can manage.
One useful way is the `value_counts()` method which you can think of as reminiscent of a histogram.
For each time difference that is in the array, the computer will output how many times that difference occurred.

In [3]:
time_deltas.value_counts()

0 days 00:15:00    29676
0 days 00:30:00      546
0 days 00:45:00       13
0 days 01:00:00        5
0 days 02:30:00        3
0 days 01:45:00        3
0 days 01:15:00        3
0 days 05:15:00        2
0 days 03:15:00        1
0 days 18:30:00        1
0 days 19:15:00        1
0 days 11:15:00        1
2 days 05:30:00        1
0 days 17:00:00        1
0 days 10:45:00        1
3 days 00:00:00        1
0 days 02:00:00        1
0 days 04:30:00        1
0 days 03:45:00        1
0 days 07:00:00        1
0 days 02:15:00        1
2 days 01:00:00        1
0 days 01:30:00        1
1 days 06:45:00        1
Name: DT, dtype: int64

Now, we want to only consider the time differences greater than a certain size since these are the important lapses in data.
For this example, we restrict our interest to differences greater than thirty minutes and use value_counts to make a list.

In [4]:
# filter out short time_deltas
time_deltas[time_deltas > '00:30:00'].value_counts()

0 days 00:45:00    13
0 days 01:00:00     5
0 days 01:15:00     3
0 days 01:45:00     3
0 days 02:30:00     3
0 days 05:15:00     2
0 days 11:15:00     1
0 days 03:15:00     1
0 days 19:15:00     1
0 days 02:00:00     1
0 days 18:30:00     1
1 days 06:45:00     1
0 days 07:00:00     1
0 days 01:30:00     1
0 days 17:00:00     1
0 days 02:15:00     1
2 days 05:30:00     1
2 days 01:00:00     1
0 days 10:45:00     1
0 days 04:30:00     1
3 days 00:00:00     1
0 days 03:45:00     1
Name: DT, dtype: int64

To quantify this, we can use the `sum()` method on this filtered list.
This will give us the total time that has large gaps of data.

In [5]:
# sum up time_deltas
time_deltas[time_deltas > '00:30:00'].sum()

Timedelta('14 days 12:00:00')